In [1]:
# === SEKCJA 1: IMPORT I WCZYTANIE DANYCH ===
import pandas as pd
import numpy as np
import re
from IPython.display import display
import pickle
import os

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

print("--- Wczytywanie danych ---")
try:
    df_main_raw = pd.read_csv('saleflats_mazowieckie_c.csv', sep=',', header=None, on_bad_lines='skip', low_memory=False)
    # Wczytujemy nasz nowy, zweryfikowany słownik
    df_slownik = pd.read_csv('slownik_finalny_z_hierarchia.csv', sep=';')
    print("Pliki wczytane pomyślnie.")
except FileNotFoundError as e:
    print(f"BŁĄD: Nie znaleziono pliku: {e.filename}.")
    raise

C:\Users\Ai\AppData\Local\miniconda3\envs\projekt_stan\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


--- Wczytywanie danych ---
Pliki wczytane pomyślnie.


In [2]:
# === SEKCJA 2 (v10): PRZYGOTOWANIE DANYCH DLA MODELU PRZEWIDUJĄCEGO TYLKO ULICĘ ===
print("--- Przygotowanie danych do modelu ---\n")

# --- 2.1: Przygotowanie danych z ofert ---
df_main = df_main_raw.copy()
df_main.columns = [i for i in range(53)] + ['WojewodztwoID', 'PowiatID', 'GminaID', 'RodzajGminyID', 'MiastoID', 'DzielnicaID', 'UlicaID']

main_cols_map = {
    0: 'SaleId', 3: 'Title', 4: 'Description', 5: 'Area', 6: 'Price',
    12: 'BuiltYear', 17: 'NumberOfRooms', 35: 'Floor', 36: 'Floors'
}
df_main.rename(columns=main_cols_map, inplace=True)

numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear']
text_features = ['Title', 'Description']
id_features = ['UlicaID']

for col in numeric_features + id_features:
    df_main[col] = pd.to_numeric(df_main[col], errors='coerce')
for col in text_features:
    df_main[col] = df_main[col].fillna('')

df_main.dropna(subset=numeric_features + id_features, inplace=True)
df_main['UlicaID'] = df_main['UlicaID'].astype(int)

# --- 2.2: Łączenie ofert ze słownikiem ---
print("\nŁączenie ofert z danymi ze słownika...")
df_merged = pd.merge(df_main, df_slownik, on='UlicaID', how='inner')
print(f"Liczba ofert po połączeniu ze słownikiem: {len(df_merged)}")
if len(df_merged) == 0: raise ValueError("Połączenie danych nie dało żadnych wyników.")

df_model_ready = df_merged.copy()
print(f"Finalny zbiór danych gotowy. Wiersze: {len(df_model_ready)}")

# --- 2.3: Przygotowanie Danych Wejściowych (X) ---
# Wzbogacamy opis o tytuł ORAZ o PRAWDZIWĄ nazwę dzielnicy i ulicy, aby model uczył się kontekstu
print("\nWzbogacanie opisów o tytuł i nazwy lokalizacji w celu wzmocnienia sygnału...")
df_model_ready['description_enriched'] = df_model_ready['Title'] + " " + df_model_ready['Description'] + " " + df_model_ready['Dzielnica_Name'] + " " + df_model_ready['Ulica_Name']
def clean_text(text): return re.sub(r'[^a-ząęółśżźćń ]', '', str(text).lower())
df_model_ready['description_clean'] = df_model_ready['description_enriched'].apply(clean_text)

# Tokenizacja
MAX_WORDS, MAX_LEN = 20000, 250
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<unk>")
tokenizer.fit_on_texts(df_model_ready['description_clean'])
X_text = pad_sequences(tokenizer.texts_to_sequences(df_model_ready['description_clean']), maxlen=MAX_LEN)

# Cechy numeryczne
df_model_ready['Price_per_sqm'] = df_model_ready['Price'] / df_model_ready['Area']
df_model_ready['Price_per_sqm'].replace([np.inf, -np.inf], np.nan, inplace=True)
numeric_features_cols = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear', 'Price_per_sqm']
numeric_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
X_numeric = numeric_pipeline.fit_transform(df_model_ready[numeric_features_cols])

# --- 2.4: Przygotowanie Danych Wyjściowych (y) ---
# === ZMIANA: Naszym jedynym celem jest teraz ulica ===
le_ulica = LabelEncoder()
y_ulica = le_ulica.fit_transform(df_model_ready['Ulica_Name'])
num_classes_ulica = len(le_ulica.classes_)
# Zachowujemy koder dla dzielnic do późniejszego użytku, ale nie trenujemy na nim
le_dzielnica = LabelEncoder()
le_dzielnica.fit(df_model_ready['Dzielnica_Name'])

print(f"\nProblem przygotowany do modelowania:")
print(f" - Cel: Predykcja ulicy")
print(f" - Liczba klas (ulice): {num_classes_ulica}")

# Dzielimy dane, stratyfikując po DZIELNICY, aby zapewnić zrównoważony rozkład geograficzny w obu zbiorach
train_indices, val_indices = train_test_split(
    range(len(df_model_ready)),
    test_size=0.2,
    random_state=42,
    stratify=df_model_ready['Dzielnica_Name'] # Stratyfikacja po dzielnicy
)

X_train_text, X_val_text = X_text[train_indices], X_text[val_indices]
X_train_num, X_val_num = X_numeric[train_indices], X_numeric[val_indices]
y_train_ulica, y_val_ulica = y_ulica[train_indices], y_ulica[val_indices]

print("\nDane podzielone na zbiory treningowe i walidacyjne.")

--- Przygotowanie danych do modelu ---


Łączenie ofert z danymi ze słownika...
Liczba ofert po połączeniu ze słownikiem: 8066
Finalny zbiór danych gotowy. Wiersze: 8066

Wzbogacanie opisów o tytuł i nazwy lokalizacji w celu wzmocnienia sygnału...

Problem przygotowany do modelowania:
 - Cel: Predykcja ulicy
 - Liczba klas (ulice): 661

Dane podzielone na zbiory treningowe i walidacyjne.


In [3]:
# === SEKCJA 3 (v10): BUDOWA I TRENING UPROSZCZONEGO MODELU (TYLKO ULICA) ===

# --- 3.1: Definicja architektury ---
input_text = Input(shape=(MAX_LEN,), name='text_input')
input_numeric = Input(shape=(X_numeric.shape[1],), name='numeric_input')

# Wspólny trzon
text_embedding = Embedding(input_dim=MAX_WORDS, output_dim=128)(input_text)
lstm_out = LSTM(128, dropout=0.3)(text_embedding)
concatenated = Concatenate()([lstm_out, input_numeric])

# Wspólna część gęsta
x = Dense(256, activation='relu')(concatenated)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# === ZMIANA: Tylko jedno wyjście dla ulicy ===
ulica_output = Dense(num_classes_ulica, activation='softmax', name='output_ulica')(x)

# --- 3.2: Kompilacja modelu ---
model = Model(inputs=[input_text, input_numeric], outputs=[ulica_output])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Pojedyncza funkcja straty
    metrics=['accuracy'] # Pojedyncza metryka
)
model.summary()

# --- 3.3: Trening ---
# Zestawy danych są teraz prostsze
X_train = [X_train_text, X_train_num]
y_train = y_train_ulica
X_val = [X_val_text, X_val_num]
y_val = y_val_ulica

callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)
]

print("\nRozpoczynam trening modelu do predykcji ulic...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=128,
    callbacks=callbacks
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 250)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 250, 128)          │       2,560,000 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 128)               │         131,584 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ numeric_input (InputLayer)    │ (None, 7)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 135)               │               0 │ lstm[0][0],                │
│                               │                           │                 │ numeric_input[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 256)               │          34,816 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 256)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 128)               │          32,896 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 128)               │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_ulica (Dense)          │ (None, 661)               │          85,269 │ dropout_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,844,565 (10.85 MB)

 Trainable params: 2,844,565 (10.85 MB)

 Non-trainable params: 0 (0.00 B)


Rozpoczynam trening modelu do predykcji ulic...
Epoch 1/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 22s 378ms/step - accuracy: 0.0495 - loss: 6.1479 - val_accuracy: 0.1413 - val_loss: 4.8904 - learning_rate: 0.0010
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 18s 359ms/step - accuracy: 0.1421 - loss: 4.6560 - val_accuracy: 0.2336 - val_loss: 4.0448 - learning_rate: 0.0010
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 18s 361ms/step - accuracy: 0.2471 - loss: 3.8800 - val_accuracy: 0.3265 - val_loss: 3.4805 - learning_rate: 0.0010
Epoch 4/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 18s 356ms/step - accuracy: 0.3146 - loss: 3.4037 - val_accuracy: 0.4058 - val_loss: 3.0533 - learning_rate: 0.0010
Epoch 5/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 18s 357ms/step - accuracy: 0.3757 - loss: 2.9951 - val_accuracy: 0.4684 - val_loss: 2.6955 - learning_rate: 0.0010
Epoch 6/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 18s 355ms/step - accuracy: 0.4245 - loss: 2.6560 - val_accuracy: 0.5477 - val_loss: 2.3633 - learning_rate: 0.0010
Epoch 7/50
51/51 ━━━━━━━━━━━━━━━━

In [4]:
# === SEKCJA 4 (v10): ZAPIS ARTEFAKTÓW DO PRODUKCJI ===
artifacts_dir = 'model_artifacts_final_v10' # Nowy folder, aby nie nadpisać starego
os.makedirs(artifacts_dir, exist_ok=True)

# 1. Zapis modelu
model.save(os.path.join(artifacts_dir, 'street_predictor_model.keras'))

# 2. Zapis Tokenizera
with open(os.path.join(artifacts_dir, 'tokenizer.pkl'), 'wb') as f: pickle.dump(tokenizer, f)

# 3. Zapis pipelinu numerycznego
with open(os.path.join(artifacts_dir, 'numeric_pipeline.pkl'), 'wb') as f: pickle.dump(numeric_pipeline, f)

# 4. Zapis koderów
with open(os.path.join(artifacts_dir, 'le_dzielnica.pkl'), 'wb') as f: pickle.dump(le_dzielnica, f)
with open(os.path.join(artifacts_dir, 'le_ulica.pkl'), 'wb') as f: pickle.dump(le_ulica, f)
    
# 5. Zapis słownika (dla wygody)
df_slownik.to_csv(os.path.join(artifacts_dir, 'slownik.csv'), index=False)

print(f"\nWszystkie artefakty zostały zapisane w folderze: '{artifacts_dir}'")


Wszystkie artefakty zostały zapisane w folderze: 'model_artifacts_final_v10'


In [6]:
# === SEKCJA 5 (v10.1): ZAAWANSOWANA PREdykcja (ULEPSZONE REGUŁY + MODEL) ===

import pandas as pd
import numpy as np
import pickle
import os
import re
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from IPython.display import display

print("--- Rozpoczynam proces predykcji hybrydowej (z zaawansowanymi regułami) ---")

# --- 5.1: Wczytanie zapisanych artefaktów ---
artifacts_dir = 'model_artifacts_final_v10'
try:
    print(f"Wczytuję artefakty z folderu: '{artifacts_dir}'...")
    model = load_model(os.path.join(artifacts_dir, 'street_predictor_model.keras'))
    tokenizer = pickle.load(open(os.path.join(artifacts_dir, 'tokenizer.pkl'), 'rb'))
    numeric_pipeline = pickle.load(open(os.path.join(artifacts_dir, 'numeric_pipeline.pkl'), 'rb'))
    le_dzielnica = pickle.load(open(os.path.join(artifacts_dir, 'le_dzielnica.pkl'), 'rb'))
    le_ulica = pickle.load(open(os.path.join(artifacts_dir, 'le_ulica.pkl'), 'rb'))
    df_slownik = pd.read_csv(os.path.join(artifacts_dir, 'slownik.csv'))
    print("Artefakty wczytane pomyślnie.")
except Exception as e:
    print(f"BŁĄD: {e}")
    raise

# --- ETAP 1: Predykcja Dzielnicy (ULEPSZONE REGUŁY) ---
print("\nEtap 1: Wykrywanie dzielnicy z tytułu za pomocą zaawansowanych reguł...")

# Słownik mapujący różne warianty na oficjalną nazwę. Można go łatwo rozbudowywać.
DISTRICT_MAPPING = {
    # Bemowo
    'bemowo': 'Bemowo', 'bemowie': 'Bemowo',
    # Białołęka
    'białołęka': 'Białołęka', 'białołęce': 'Białołęka',
    # Bielany
    'bielany': 'Bielany', 'bielanach': 'Bielany',
    # Mokotów
    'mokotów': 'Mokotów', 'mokotowie': 'Mokotów',
    # Ochota
    'ochota': 'Ochota', 'ochocie': 'Ochota',
    # Praga-Południe
    'praga-południe': 'Praga-południe', 'pradze-południe': 'Praga-południe',
    'praga południe': 'Praga-południe', 'praga płd': 'Praga-południe',
    # Praga-Północ
    'praga-północ': 'Praga-północ', 'pradze-północ': 'Praga-północ',
    'praga północ': 'Praga-północ', 'praga płn': 'Praga-północ',
    # Śródmieście
    'śródmieście': 'Śródmieście', 'śródmieściu': 'Śródmieście',
    # Targówek
    'targówek': 'Targówek', 'targówku': 'Targówek',
    # Ursus
    'ursus': 'Ursus', 'ursusie': 'Ursus',
    # Ursynów
    'ursynów': 'Ursynów', 'ursynowie': 'Ursynów',
    # Wawer
    'wawer': 'Wawer', 'wawrze': 'Wawer',
    # Wesoła
    'wesoła': 'Wesoła', 'wesołej': 'Wesoła',
    # Wilanów
    'wilanów': 'Wilanów', 'wilanowie': 'Wilanów',
    # Włochy
    'włochy': 'Włochy', 'włochach': 'Włochy',
    # Wola
    'wola': 'Wola', 'woli': 'Wola',
    # Żoliborz
    'żoliborz': 'Żoliborz', 'żoliborzu': 'Żoliborz',
}

# Sortujemy klucze od najdłuższego do najkrótszego, aby najpierw dopasować np. "praga-południe" a nie "praga"
sorted_district_keys = sorted(DISTRICT_MAPPING.keys(), key=len, reverse=True)

def find_district_in_title_advanced(title, mapping, sorted_keys):
    title_lower = str(title).lower()
    for key in sorted_keys:
        # Używamy \b (granica słowa), aby uniknąć dopasowań w środku słów (np. "Wola" w "Wolałbym")
        if re.search(r'\b' + re.escape(key) + r'\b', title_lower):
            return mapping[key] # Zwracamy oficjalną nazwę
    return None

# --- Aplikujemy nową logikę ---
df_to_predict = df_main_raw.copy()
main_cols_map = {3: 'Title'}
df_to_predict.rename(columns=main_cols_map, inplace=True)
df_to_predict['predicted_dzielnica'] = df_to_predict['Title'].apply(
    lambda title: find_district_in_title_advanced(title, DISTRICT_MAPPING, sorted_district_keys)
)

found_count = df_to_predict['predicted_dzielnica'].notna().sum()
print(f"Znaleziono dzielnicę w tytule dla {found_count} z {len(df_to_predict)} ogłoszeń. (Poprzednio: 91670)")


# --- Przygotowanie danych do Etapu 2 ---
# (Reszta kodu pozostaje identyczna jak w poprzedniej wersji v10)
df_to_predict = df_main_raw.copy()
full_main_cols_map = {
    0: 'SaleId', 3: 'Title', 4: 'Description', 5: 'Area', 6: 'Price',
    12: 'BuiltYear', 17: 'NumberOfRooms', 35: 'Floor', 36: 'Floors'
}
df_to_predict.rename(columns=full_main_cols_map, inplace=True)

text_features = ['Title', 'Description']
numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear']
for col in text_features: df_to_predict[col] = df_to_predict[col].fillna('')
for col in numeric_features: df_to_predict[col] = pd.to_numeric(df_to_predict[col], errors='coerce')
    
df_valid_for_pred = df_to_predict.dropna(subset=numeric_features).copy()
df_valid_for_pred['Price_per_sqm'] = df_valid_for_pred['Price'] / df_valid_for_pred['Area']
df_valid_for_pred['Price_per_sqm'].replace([np.inf, -np.inf], np.nan, inplace=True)

df_valid_for_pred['predicted_dzielnica'] = df_valid_for_pred['Title'].apply(
    lambda title: find_district_in_title_advanced(title, DISTRICT_MAPPING, sorted_district_keys)
)
df_valid_for_pred['description_enriched'] = df_valid_for_pred['Title'] + " " + df_valid_for_pred['Description'] + " " + df_valid_for_pred['predicted_dzielnica'].fillna('')
def clean_text(text): return re.sub(r'[^a-ząęółśżźćń ]', '', str(text).lower())
df_valid_for_pred['description_clean'] = df_valid_for_pred['description_enriched'].apply(clean_text)

MAX_LEN = 250
X_text_pred = pad_sequences(tokenizer.texts_to_sequences(df_valid_for_pred['description_clean']), maxlen=MAX_LEN)
numeric_features_cols = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear', 'Price_per_sqm']
X_numeric_pred = numeric_pipeline.transform(df_valid_for_pred[numeric_features_cols])

# --- ETAP 2: Predykcja Ulicy (Model ML) ---
print("\nEtap 2: Predykcja ulicy za pomocą modelu...")
pred_ulica_probs = model.predict([X_text_pred, X_numeric_pred])

# --- Korekta Wyników ---
print("Korekta wyników, aby ulica należała do przewidzianej dzielnicy...")
dzielnica_to_ulice_map = df_slownik.groupby('Dzielnica_Name')['Ulica_Name'].apply(list).to_dict()
ulica_name_to_idx_map = {name: i for i, name in enumerate(le_ulica.classes_)}

corrected_ulica_indices = []
for i, row in df_valid_for_pred.iterrows():
    dzielnica = row['predicted_dzielnica']
    all_ulica_probs = pred_ulica_probs[len(corrected_ulica_indices)]
    
    if pd.isna(dzielnica):
        best_ulica_idx = np.argmax(all_ulica_probs)
    else:
        valid_ulica_names = dzielnica_to_ulice_map.get(dzielnica, [])
        valid_ulica_indices = [ulica_name_to_idx_map.get(name) for name in valid_ulica_names]
        valid_ulica_indices = [idx for idx in valid_ulica_indices if idx is not None]

        if not valid_ulica_indices:
            best_ulica_idx = np.argmax(all_ulica_probs)
        else:
            valid_probs = all_ulica_probs[valid_ulica_indices]
            best_local_idx = np.argmax(valid_probs)
            best_ulica_idx = valid_ulica_indices[best_local_idx]
            
    corrected_ulica_indices.append(best_ulica_idx)

df_valid_for_pred['predicted_ulica_final'] = le_ulica.inverse_transform(corrected_ulica_indices)

# --- 5.6: Wyświetlenie wyników ---
print("\nPrzykładowe wyniki predykcji hybrydowej (z zaawansowanymi regułami):")
display(df_valid_for_pred[[
    'Title', 'predicted_dzielnica', 'predicted_ulica_final'
]].head(30))

--- Rozpoczynam proces predykcji hybrydowej (z zaawansowanymi regułami) ---
Wczytuję artefakty z folderu: 'model_artifacts_final_v10'...
Artefakty wczytane pomyślnie.

Etap 1: Wykrywanie dzielnicy z tytułu za pomocą zaawansowanych reguł...
Znaleziono dzielnicę w tytule dla 103561 z 235700 ogłoszeń. (Poprzednio: 91670)

Etap 2: Predykcja ulicy za pomocą modelu...
5610/5610 ━━━━━━━━━━━━━━━━━━━━ 437s 78ms/step
Korekta wyników, aby ulica należała do przewidzianej dzielnicy...

Przykładowe wyniki predykcji hybrydowej (z zaawansowanymi regułami):


,Title,predicted_dzielnica,predicted_ulica_final
0,"Mieszkanie, Mokotów",Mokotów,Tawerny
1,Dwupoziomowy apartament w doskonałej lokalizacji,None,Nowosielecka
2,Bezpośrednio! mieszkanie - Wilanów,Wilanów,Kazachska
4,"Mieszkanie 3-pokoje, umeblowane, po remoncie",None,Aleja tysiąclecia
5,PREMIUM !!! Apartament z Widokiem 22Piętro!Taras!,None,Béli bartóka
6,"Mieszkanie Bródno, ul. Wysockiego",None,Oskara sosnowskiego
7,Moja Północna II | mieszkanie B 3,None,Stanisława worcella
12,"Mieszkanie Warszawa Śródmieście, ul. Jana Pawł...",Śródmieście,Klonowa
14,3 pokoje Mokotów Woronicza/Suwak,Mokotów,Tawerny
15,Mieszkanie trzypokojowe na sprzedaż,None,Andyjska
